In [22]:
import os
import importlib
import melee 
import numpy as np
import pandas as pd
import data_processing 
importlib.reload(data_processing) 
from data_processing import process_slp_file, save_processed_data, process_and_save_sequences, process_archive_files_streaming, batch_process_archives, process_directory_slp_files

In [23]:
os.getcwd()

'/Users/alexoon/Desktop/smash_bot/icl_smash'

In [24]:
# # Process a single file
# slp_file = "./data/Stream-Game_20220828T223051.slp"
# sampled_data, columns = process_slp_file(slp_file)
# sampled_data.shape

# # Create and save sequences
# input_seqs, target_seqs = process_and_save_sequences(
#     sampled_data, 
#     "./training_data", 
#     "[libmeleeV2]example1",
#     input_len=10,
#     target_len=5
# )
# # Save the results
# # save_processed_data(sampled_data, columns, "./training_data", "[libmeleeV2]training_ex1.npy", save_csv=True)

# # Or use the main function directly
# # shape = main(slp_file, "./training_data", "DESIRED_FILENAME")

In [25]:
# Define input and output directories
data_dir = "./data"
output_dir = "./training_data"

# Make sure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process all .slp files
processed_count, failed = process_directory_slp_files(
    directory_path=data_dir,
    output_dir=output_dir,
    input_len=10,
    target_len=5
)

Found 3 .slp files to process


Processing .slp files:  33%|███▎      | 1/3 [00:01<00:02,  1.23s/it]

Processed slp file shape: (10549, 59)
Saved sequences to ./training_data/(...) Falco vs Falco (BABE) [DL] Game_20200122T213501_sequences.npz
Input shape: (690, 10, 59)
Target shape: (690, 5, 59)


Processing .slp files:  67%|██████▋   | 2/3 [00:02<00:01,  1.01s/it]

Processed slp file shape: (7106, 59)
Saved sequences to ./training_data/(...) Falco vs Falcon (L$) [YS] Game_20200226T210843_sequences.npz
Input shape: (460, 10, 59)
Target shape: (460, 5, 59)


Processing .slp files: 100%|██████████| 3/3 [00:03<00:00,  1.03s/it]

Processed slp file shape: (9545, 59)
Saved sequences to ./training_data/(...) Falco vs Falco (BABE) [BF] Game_20200122T213211_sequences.npz
Input shape: (623, 10, 59)
Target shape: (623, 5, 59)

Successfully processed 3 files
Results saved to: ./training_data
